In [10]:
import openpyxl   
import pandas as pd
import numpy as np
from pandas import read_excel
import matplotlib.pyplot as plt
import scipy.optimize as scp

In [24]:
number = len(test_samples)-2
test_samples = ['nitrogen','air1','air2','crone1','crone2','crone3','spark1','spark2']
raw_data=pd.read_excel(str(test_samples[number]+".xlsx"), engine = 'openpyxl') 
data = pd.DataFrame(raw_data)
data = data.drop([0,1]).reset_index(drop=True)
data

,Unnamed: 0,Sample,T [s],ch1 [V],ch2 [V],mean [V],inverted [V]
0,2,-329,-0.000007,0.000801,0.000297,0.000279,-0.000279
1,3,-328,-0.000007,0.000801,0.000297,0.000259,-0.000259
2,4,-327,-0.000007,0.000701,0.000496,0.000294,-0.000294
3,5,-326,-0.000007,0.000701,0.000397,0.000260,-0.000260
4,6,-325,-0.000006,0.001202,0.000198,0.000270,-0.000270
...,...,...,...,...,...,...,...
131065,131067,130736,0.002615,0.000801,0.000198,0.000257,-0.000257
131066,131068,130737,0.002615,0.000901,0.000198,0.000287,-0.000287
131067,131069,130738,0.002615,0.001102,0.000000,0.000276,-0.000276
131068,131070,130739,0.002615,0.000901,0.000496,0.000272,-0.000272


In [25]:
time_column = data.columns[2]
ch1_column = data.columns[3]
data_column = data.columns[-1]
print(time_column,ch1_column,data_column)
#print(np.average(data[ch1_column])*10)

%matplotlib widget
plt.plot(data[time_column],data[ch1_column])
#plt.set_size_inches(10, 4)
plt.xlabel("czas [s]")
plt.title("ch1/ch2 (niebieski/czerwony)")
plt.plot(data[time_column],data[data_column], color='r')
plt.show()


T [s] ch1 [V] inverted [V]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
tester = []
for i in range(len(data[ch1_column])):
    tester.append(data[ch1_column].to_numpy()[i]-data[ch1_column].to_numpy()[i-1])

thershold =  (np.max(tester)-np.average(tester))/3.5
print(thershold)

%matplotlib widget
plt.plot(data[time_column], tester)
plt.axhline(thershold, color='r')
plt.xlabel("czas [s]")
plt.title("przetworzony sygnał ch1")
#plt.plot(data[time_column],data[data_column], color='r')

0.004835825285714286


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'przetworzony sygnał ch1')

In [14]:
def f(x,a,b,c):
    return ((a*np.exp(-x/b))-c)
    

In [27]:
points_candidates = np.argwhere(tester>thershold)
points_candidates = np.append(points_candidates,[[(len(data[ch1_column])-1)]], axis=0)
#points_candidates.append()

i = 0
points = []
while i<(len(points_candidates)-1):
    distance = points_candidates[i+1][0]-points_candidates[i][0]
    if distance < 200:
        if data[ch1_column][points_candidates[i][0]]>data[ch1_column][points_candidates[i+1][0]]:
            points.append(points_candidates[i][0])
        else:
            points.append(points_candidates[i+1][0])
        i = i + 1

    else:
        points.append(points_candidates[i][0])

    i = i + 1

points.append(len(data[ch1_column])-1)
#print(points)
#print(points_candidates)


%matplotlib widget
plt.plot(data[time_column],data[data_column])


for i in range(len(points)):
    plt.axvline(x= data[time_column][points[i]], color = 'r')

"""

for i in range(len(points_candidates)):
    plt.axvline(x= data[time_column][points_candidates[i][0]], color = 'g')

"""

plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
int_indexes = np.arange(points[0],points[1])
temp_data = data[data_column].to_numpy()
average = temp_data[int_indexes]
res = []

for i in range(len(int_indexes)):
    temp = []
    for k in range(len(points)):
        try:
            temp.append(temp_data[points[k]+i])
        except:
            pass
    res.append(np.average(temp))


"""
for i in range(1,len(points)-1):
    indexes = np.arange(points[i],points[i+1])
    for k in range(min(len(average),len(indexes))):
        average[k] = average[k] + data[data_column].to_numpy()[indexes[k]]
"""
%matplotlib widget
plt.plot(data[time_column].to_numpy()[int_indexes], res)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
peak = np.argmax(res)

t_0 = data[time_column ].to_numpy()[int_indexes][peak:][0]
times_fit = np.array(data[time_column ].to_numpy()[int_indexes][peak:] - t_0, dtype=np.float32)
#times_fit = data[time_column ].to_numpy()[int_indexes][peak:]
val_fit = res[peak:]

a_init = val_fit[0]
b_init = 0.00000014
c_init = -0.0002

params, pcov = scp.curve_fit(f, times_fit, val_fit, [a_init,b_init,c_init])
err = np.sqrt(np.diag(pcov))
print(params, err)

%matplotlib widget
plt.plot(data[time_column].to_numpy()[int_indexes], res)
plt.plot((times_fit+t_0), f(times_fit, *params), color = 'r')
plt.xlabel("czas [s]")
plt.ylabel("napięcie [V]")
plt.title(test_samples[number])
plt.show()



[3.80484858e-04 4.57535002e-07 2.65097590e-04] [1.77645652e-06 3.10264334e-09 6.53005124e-08]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

nitrogen:
2.3585663246765498e-06 6.909764575890499e-08
air1:
[1.17512878e-03      2.15269798e-04] [1.04873271e-06 3.05309210e-09 8.81212020e-08]
air2:
2.3452365742603783e-06 5.3840971863432014e-08
crone1:
1.2384908420188277e-06 8.084168262122479e-08
crone2:
5.165562047605518e-07 2.63762452029122e-08
crone3:
2.0042874543094457e-06 5.207545170862822e-08
spark1:
2.2626358380928828e-07 9.484269041802186e-08
spark2:
1.4553462809515665e-07 2.6910637686926914e-08


